In [1]:
from pathlib import Path
import os
import sys

definitions_path = Path(sys.prefix) / "share/eccodes-cosmo-resources/definitions"
os.environ["ECCODES_DEFINITION_PATH"] = str(definitions_path)

import dask
import earthkit.data as ekd
import numpy as np
from obstore.store import LocalStore
import xarray as xr
from virtualizarr.registry import ObjectStoreRegistry
from virtualizarr import open_virtual_dataset

from icon_ch_vzarr import IconChParser
from icon_ch_vzarr.parsers import DEFAULT_PROFILE

In [17]:
# FILTERS = {"shortName": ["T", "T_2M","W","CLCL","TOT_PREC"]}
FILTERS = {"shortName": ["T_2M","CLCL","TOT_PREC"]}

FILENAME = "/store_new/mch/msopr/osm/KENDA-CH1/ANA25/det/"
files = sorted(Path(FILENAME).glob("iaf202501012*"))
urls = [f"file://{file}" for file in files]

print(f"Found {len(urls)} files")

Found 4 files


In [18]:
source = ekd.from_source("file", files).sel(FILTERS)
ds = source.to_xarray(**DEFAULT_PROFILE)
ds

<xarray.Dataset> Size: 110MB
Dimensions:     (valid_time: 4, values: 1147980)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 32B 2025-01-01T20:00:00 ... 2025-...
Dimensions without coordinates: values
Data variables:
    CLCL        (valid_time, values) float64 37MB ...
    TOT_PREC    (valid_time, values) float64 37MB ...
    T_2M        (valid_time, values) float64 37MB ...
Attributes:
    date:         20250101
    time:         2000
    Conventions:  CF-1.8
    institution:  MeteoSwiss

In [19]:
store = LocalStore("/")
registry = ObjectStoreRegistry({"file://": store})
parser = IconChParser(filter_keys=FILTERS)

manifest_store = parser(urls[0], registry)
ds = xr.open_zarr(manifest_store, consolidated=False, decode_times=False, zarr_format=3)
ds

data=MaskIndex(MaskIndex(GRIBReader(/store_new/mch/msopr/osm/KENDA-CH1/ANA25/det/iaf2025010120),[1284, 1437, 1473]),[0, 1, 2]) 3
data=MaskIndex(MaskIndex(MaskIndex(MaskIndex(GRIBReader(/store_new/mch/msopr/osm/KENDA-CH1/ANA25/det/iaf2025010120),[1284, 1437, 1473]),[0, 1, 2]),[0]),[0]) 1
data=MaskIndex(MaskIndex(MaskIndex(MaskIndex(GRIBReader(/store_new/mch/msopr/osm/KENDA-CH1/ANA25/det/iaf2025010120),[1284, 1437, 1473]),[0, 1, 2]),[1]),[0]) 1
data=MaskIndex(MaskIndex(MaskIndex(MaskIndex(GRIBReader(/store_new/mch/msopr/osm/KENDA-CH1/ANA25/det/iaf2025010120),[1284, 1437, 1473]),[0, 1, 2]),[2]),[0]) 1


<xarray.Dataset> Size: 28MB
Dimensions:     (valid_time: 1, values: 1147980)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 8B 2025-01-01T20:00:00
Dimensions without coordinates: values
Data variables:
    T_2M        (valid_time, values) float64 9MB dask.array<chunksize=(1, 1147980), meta=np.ndarray>
    CLCL        (valid_time, values) float64 9MB dask.array<chunksize=(1, 1147980), meta=np.ndarray>
    TOT_PREC    (valid_time, values) float64 9MB dask.array<chunksize=(1, 1147980), meta=np.ndarray>

In [38]:
from pprint import pprint
pprint(manifest_store._group.arrays["T_2M"].metadata)

ArrayV3Metadata(shape=(1, 1147980),
                data_type=Float64(endianness='little'),
                chunk_grid=RegularChunkGrid(chunk_shape=(1, 1147980)),
                chunk_key_encoding=DefaultChunkKeyEncoding(name='default',
                                                           separator='/'),
                fill_value=np.float64(0.0),
                codecs=(EccodesCodec(),),
                attributes={'_earthkit': {'b64message': 'R1JJQv//AAIAAAAAAAAAqgAAABUBANcA/w8BAQfpAQEUAAABAQAAABwCAP4AB+kBARU0AwAAAAAAAAAAAAAAAAEAAAAjAwAAEYRMAAAAZQYAAAEBF2Q9oldJWbZE0lSjzW4rwAAAACIEAAAAAAAAAgCXAAAAAAAAAABnAAAAAAL///////8AAAAVBQAAAfAAAEOIgACACgAAAAAAAAAGBv8AAAAFBzc3Nzc=',
                                          'bitsPerValue': 16},
                            'long_name': '2m Temperature',
                            'standard_name': 'air_temperature',
                            'units': 'K'},
                dimension_names=('valid_time', 'values'),
                zarr_forma

In [20]:
ds["T_2M"].values

array([[275.73461914, 275.84399414, 275.67114258, ..., 275.83374023,
        275.7800293 , 275.82543945]], shape=(1, 1147980))

In [21]:
tasks = [dask.delayed(open_virtual_dataset)(url, registry, parser) for url in urls]
vds = dask.compute(*tasks, scheduler="processes")
ds = xr.combine_by_coords(vds, combine_attrs="override")
ds

data=MaskIndex(MaskIndex(GRIBReader(/store_new/mch/msopr/osm/KENDA-CH1/ANA25/det/iaf2025010120),[1284, 1437, 1473]),[0, 1, 2]) 3
data=MaskIndex(MaskIndex(MaskIndex(MaskIndex(GRIBReader(/store_new/mch/msopr/osm/KENDA-CH1/ANA25/det/iaf2025010120),[1284, 1437, 1473]),[0, 1, 2]),[0]),[0]) 1
data=MaskIndex(MaskIndex(MaskIndex(MaskIndex(GRIBReader(/store_new/mch/msopr/osm/KENDA-CH1/ANA25/det/iaf2025010120),[1284, 1437, 1473]),[0, 1, 2]),[1]),[0]) 1
data=MaskIndex(MaskIndex(MaskIndex(MaskIndex(GRIBReader(/store_new/mch/msopr/osm/KENDA-CH1/ANA25/det/iaf2025010120),[1284, 1437, 1473]),[0, 1, 2]),[2]),[0]) 1
data=MaskIndex(MaskIndex(GRIBReader(/store_new/mch/msopr/osm/KENDA-CH1/ANA25/det/iaf2025010123),[1284, 1437, 1473]),[0, 1, 2]) 3
data=MaskIndex(MaskIndex(MaskIndex(MaskIndex(GRIBReader(/store_new/mch/msopr/osm/KENDA-CH1/ANA25/det/iaf2025010123),[1284, 1437, 1473]),[0, 1, 2]),[0]),[0]) 1
data=MaskIndex(MaskIndex(MaskIndex(MaskIndex(GRIBReader(/store_new/mch/msopr/osm/KENDA-CH1/ANA25/det/iaf20

<xarray.Dataset> Size: 110MB
Dimensions:     (valid_time: 4, values: 1147980)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 32B 2025-01-01T20:00:00 ... 2025-...
Dimensions without coordinates: values
Data variables:
    T_2M        (valid_time, values) float64 37MB ManifestArray<shape=(4, 114...
    CLCL        (valid_time, values) float64 37MB ManifestArray<shape=(4, 114...
    TOT_PREC    (valid_time, values) float64 37MB ManifestArray<shape=(4, 114...

In [22]:
# TODO: persist the virtualized dataset using icechunk or kerchunk references
# could not get it to work yet

In [41]:
ds.vz.to_kerchunk("references.json", format="json")

In [15]:
xr.open_dataset("references.json", engine="kerchunk")["T"].variable.values

ValueError: When changing to a larger dtype, its size must be a divisor of the total size in bytes of the last axis of the array.